In [55]:
import pandas as pd

In [56]:
import os

# Get the current working directory
current_dir = os.getcwd()

# Get the parent directory
parent_dir = os.path.dirname(current_dir)

print("Parent Directory:", parent_dir)

Parent Directory: /gpfs/fs1/home/j/jparkin/wongkoji/final_pipeline


In [57]:
# generate a pandas dataframe from our kraken2 report, delimited by tab ('\t')
k2_report_file = "/scratch/j/jparkin/wongkoji/outputs/kraken2/pm_mtx.kreport2"
kraken_out_file = "/scratch/j/jparkin/wongkoji/outputs/kraken2/pm_mtx_kraken.out"
fastq_file_path = "/scratch/j/jparkin/wongkoji/data/pm_merged.fq"

report_df = pd.read_csv(filepath_or_buffer=k2_report_file, 
                        sep='\t',
                        names=["%_clade", "#_clade_frags", "#_taxon_frags", "class", "ncbi_taxon_id", "name"])

# Added a header explaining each column of the data.

# %_clade: Percentage of fragments covered by the clade rooted at this taxon
# #_clade_frags: Number of fragments covered by the clade rooted at this taxon
# #_taxon_frags: Number of fragments assigned directly to this taxon
# class: A rank code indicating (U)nclassified, (R)oot, (D)omain, (K)ingdom, (P)hylum, (C)lass, (O)rder, (F)amily, (G)enus, (S)pecies.
# taxon_id: The Taxonomic ID number from NCBI
# name: scientific name of taxon

# print(report_df["class"].unique())

# get only classified species who cover 0.05% of all fragments
def filter_kraken2(class_name):
    if class_name == "S":
        return "S"
    elif "S" in class_name:
        return "S"
    else:
        return class_name
# report_df = report_df[report_df['class'].isin(["S", "S1", "S2", "S3"])]
report_df = report_df[report_df['class'].isin(["S"])]

report_df = report_df[report_df['%_clade'] >= 0.05]

print("sum of clade_% fragments for classified species:", round(report_df["%_clade"].sum(),2))
print("number of species identified", len(report_df))
report_df.head(20)


sum of clade_% fragments for classified species: 67.99
number of species identified 21


,%_clade,#_clade_frags,#_taxon_frags,class,ncbi_taxon_id,name
10,4.42,52890,52881,S,47883,Pseudomonas synxantha
12,4.16,49840,49840,S,76760,Pseudomonas rhodesiae
13,4.16,49774,49774,S,76758,Pseudomonas orientalis
14,3.53,42291,42253,S,380021,Pseudomonas protegens
16,3.10,37118,36090,S,294,Pseudomonas fluorescens
26,0.08,939,939,S,1114970,Pseudomonas ogarae
49,16.23,194296,168457,S,587753,Pseudomonas chlororaphis
60,8.29,99227,62886,S,317,Pseudomonas syringae
117,2.65,31682,31682,S,78327,Pseudomonas mosselii
118,2.53,30338,29940,S,303,Pseudomonas putida


In [58]:
#TODO: extract k-mer count information corresponding to data in stdout in kraken.out
k2_output = pd.read_csv(filepath_or_buffer=kraken_out_file, 
                        sep='\t',
                        names=["is_classified", "seq_id", "taxon_id", "bp_length", "LCA_mapping"])

In [59]:
k2_output.head()
print("num reads:", len(k2_output))

num reads: 1197492


In [60]:
# Keep only reads that are in filtered classes in report_df
k2_output_filt = k2_output[k2_output["taxon_id"].isin(report_df['ncbi_taxon_id'])]
k2_output_filt = k2_output_filt[k2_output_filt["taxon_id"] != 0]
print("num reads:", len(k2_output_filt))
k2_output_filt.head()

num reads: 750143


,is_classified,seq_id,taxon_id,bp_length,LCA_mapping
0,C,lcl|NZ_CP010896.1_cds_WP_010207718.1_1-2/1,321846,199,286:10 321846:10 286:1 321846:26 286:5 321846:...
1,C,lcl|NZ_CP010896.1_cds_WP_010207718.1_1-4/1,321846,201,321846:67 2:5 321846:20 286:1 321846:5 286:69
2,C,lcl|NZ_CP010896.1_cds_WP_010207719.1_2-4/1,321846,216,321846:6 1224:5 286:29 321846:5 286:9 321846:5...
3,C,lcl|NZ_CP010896.1_cds_WP_010207719.1_2-2/1,321846,207,321846:3 286:7 1236:1 135621:2 286:2 2:5 286:3...
4,C,lcl|NZ_CP010896.1_cds_WP_021491432.1_3-2/1,321846,187,286:39 321846:3 286:4 321846:5 286:9 321846:3 ...


In [61]:
# And can create a map of taxon_id to seq_ids assigned to that taxon
seq_by_taxon = k2_output_filt.groupby("taxon_id")['seq_id'].apply(list).to_dict()
print("number of taxa in our dataset:", len(seq_by_taxon.keys()))
# List all the taxon ids
print("Taxon ID:", list(seq_by_taxon.keys()))

# List the first five sequence ids by taxon 321846
print("Taxon 321846 Sequence IDs:", seq_by_taxon[321846][0:5])

# Get the number of taxons with ID 321846 in the list
print("# Sequences in taxon 321846:", len(seq_by_taxon[321846]))

number of taxa in our dataset: 21
Taxon ID: [287, 294, 303, 317, 47883, 53408, 76758, 76759, 76760, 78327, 107445, 321846, 380021, 587753, 930166, 1114970, 2219225, 2565368, 2745498, 2842346, 2895471]
Taxon 321846 Sequence IDs: ['lcl|NZ_CP010896.1_cds_WP_010207718.1_1-2/1', 'lcl|NZ_CP010896.1_cds_WP_010207718.1_1-4/1', 'lcl|NZ_CP010896.1_cds_WP_010207719.1_2-4/1', 'lcl|NZ_CP010896.1_cds_WP_010207719.1_2-2/1', 'lcl|NZ_CP010896.1_cds_WP_021491432.1_3-2/1']
# Sequences in taxon 321846: 51026


In [62]:
import re

# function to calculate k2 assignment score
def calculate_score(df, index: int) -> float:
    """
    Calculates the confidence score of taxonomic assignment to a sequence by
    dividing the number of k-mers mapped to a taxon by the total number of 
    k-mers from that sequence.

    Args:
        index (int): an index into the kraken2 output DataFrame, where each 
            index corresponds to a sequence
    """
    tax_id = df["taxon_id"][index]
    seq_id = k2_output["seq_id"][index]
    print("Assigned ID:", tax_id)
    lca_mapping = df["LCA_mapping"][index]
    mappings = lca_mapping.split()
    print("Mappings:", mappings)
    total = 0
    match_total = 0
    # loop over LCA mappings for this particular sequence
    for mapping in mappings:
        query = fr'([0-9]*):([0-9]*)'
        match = re.match(query, mapping) # use regex match to find query mapping
        q_id = match.group(1)
        kmer_count = int(match.group(2))
        # we want to calculate the confidence score this assignment to the taxon_id
        if int(tax_id) == int(q_id):
            total += kmer_count
            match_total += kmer_count
        else:
            total += kmer_count
    score = match_total/total
    print(f"Score for sequence {seq_id} assigned to taxon {tax_id}:", score)
    return score

In [63]:
# Now we can use this function to calculate the score from any index
calculate_score(k2_output, 1)
calculate_score(k2_output, 15)

Assigned ID: 321846
Mappings: ['321846:67', '2:5', '321846:20', '286:1', '321846:5', '286:69']
Score for sequence lcl|NZ_CP010896.1_cds_WP_010207718.1_1-4/1 assigned to taxon 321846: 0.5508982035928144
Assigned ID: 321846
Mappings: ['286:48', '321846:11', '286:1', '321846:17', '286:36', '135621:2', '286:4', '1224:5', '286:2', '2:5', '1224:2', '286:2', '1236:2', '286:2', '1224:3', '1236:5', '286:19']
Score for sequence lcl|NZ_CP010896.1_cds_WP_021491428.1_9-2/1 assigned to taxon 321846: 0.1686746987951807


0.1686746987951807

In [64]:
import pandas as pd
from Bio import SeqIO

def fasta_to_dataframe(fastq_file):
    data = {
        'seq_id': [],
        'sequence': [],
        'bp_length': []
    }

    for record in SeqIO.parse(fastq_file, "fasta"):
        data['seq_id'].append(record.id)
        data['sequence'].append(str(record.seq))
        data['bp_length'].append(len(record.seq))

    df = pd.DataFrame(data)
    return df

# fasta_file_path = "/home/j/jparkin/wongkoji/data/merged_paired_mtx.fasta"
# fasta_df = fasta_to_dataframe(fasta_file_path)

# Display the fasta df
# print(fasta_df)

In [65]:
import pandas as pd
from Bio import SeqIO

def fastq_to_dataframe(fastq_file):
    data = {
        'seq_id': [],
        'sequence': [],
        'bp_length': []
    }

    for record in SeqIO.parse(fastq_file, "fastq"):
        data['seq_id'].append(record.id)
        data['sequence'].append(str(record.seq))
        data['bp_length'].append(len(record.seq))

    df = pd.DataFrame(data)
    return df

fasta_df = fastq_to_dataframe(fastq_file_path)

# Display the fasta df
fasta_df.head()

,seq_id,sequence,bp_length
0,lcl|NZ_CP010896.1_cds_WP_010207718.1_1-2/1,GAGAATGAAGAGCCGTCCCGCGACAGCTTTGATCCGATGGCAGGCG...,199
1,lcl|NZ_CP010896.1_cds_WP_010207718.1_1-4/1,AGACGACTTGGAGGGCGCAGGAGCAGGAGACGGCGCACTGACAGCC...,201
2,lcl|NZ_CP010896.1_cds_WP_010207719.1_2-4/1,TGTGCTCTATGCGCGCCGATATCGGTCAGCCGGACCGTCACCAGGT...,216
3,lcl|NZ_CP010896.1_cds_WP_010207719.1_2-2/1,CTTGCCATCAACCAGCTTGGAGGTGAAGGTGAACTCGCCGGTGGTC...,207
4,lcl|NZ_CP010896.1_cds_WP_021491432.1_3-2/1,CCCGTCAATGCGAATCTGGAACTCTCCGCCGCGATCACGGGATATC...,187


In [66]:
a = seq_by_taxon[321846]
sequences_taxon_321846 = fasta_df[fasta_df["seq_id"].isin(a)]


In [67]:
print(sequences_taxon_321846[0:5])

                                       seq_id  \
0  lcl|NZ_CP010896.1_cds_WP_010207718.1_1-2/1   
1  lcl|NZ_CP010896.1_cds_WP_010207718.1_1-4/1   
2  lcl|NZ_CP010896.1_cds_WP_010207719.1_2-4/1   
3  lcl|NZ_CP010896.1_cds_WP_010207719.1_2-2/1   
4  lcl|NZ_CP010896.1_cds_WP_021491432.1_3-2/1   

                                            sequence  bp_length  
0  GAGAATGAAGAGCCGTCCCGCGACAGCTTTGATCCGATGGCAGGCG...        199  
1  AGACGACTTGGAGGGCGCAGGAGCAGGAGACGGCGCACTGACAGCC...        201  
2  TGTGCTCTATGCGCGCCGATATCGGTCAGCCGGACCGTCACCAGGT...        216  
3  CTTGCCATCAACCAGCTTGGAGGTGAAGGTGAACTCGCCGGTGGTC...        207  
4  CCCGTCAATGCGAATCTGGAACTCTCCGCCGCGATCACGGGATATC...        187  


In [68]:
print(f"There are {len(sequences_taxon_321846)} sequences binned in taxon 321846")

There are 51026 sequences binned in taxon 321846


In [69]:
# create a merged dataframe containing sequences (ID), assigned taxon number,
fasta_df = fasta_df[fasta_df["seq_id"].isin(k2_output_filt["seq_id"])]
merged_df = pd.merge(fasta_df, k2_output[['seq_id', 'taxon_id', "LCA_mapping"]], on='seq_id', how='left')

# Remove unclassified (taxon 0) reads
merged_df = merged_df[merged_df["taxon_id"] != 0]
print("number of taxons:", len(merged_df['taxon_id'].unique()))
print("number of reads:", len(merged_df))

merged_df.head()

number of taxons: 21
number of reads: 750143


,seq_id,sequence,bp_length,taxon_id,LCA_mapping
0,lcl|NZ_CP010896.1_cds_WP_010207718.1_1-2/1,GAGAATGAAGAGCCGTCCCGCGACAGCTTTGATCCGATGGCAGGCG...,199,321846,286:10 321846:10 286:1 321846:26 286:5 321846:...
1,lcl|NZ_CP010896.1_cds_WP_010207718.1_1-4/1,AGACGACTTGGAGGGCGCAGGAGCAGGAGACGGCGCACTGACAGCC...,201,321846,321846:67 2:5 321846:20 286:1 321846:5 286:69
2,lcl|NZ_CP010896.1_cds_WP_010207719.1_2-4/1,TGTGCTCTATGCGCGCCGATATCGGTCAGCCGGACCGTCACCAGGT...,216,321846,321846:6 1224:5 286:29 321846:5 286:9 321846:5...
3,lcl|NZ_CP010896.1_cds_WP_010207719.1_2-2/1,CTTGCCATCAACCAGCTTGGAGGTGAAGGTGAACTCGCCGGTGGTC...,207,321846,321846:3 286:7 1236:1 135621:2 286:2 2:5 286:3...
4,lcl|NZ_CP010896.1_cds_WP_021491432.1_3-2/1,CCCGTCAATGCGAATCTGGAACTCTCCGCCGCGATCACGGGATATC...,187,321846,286:39 321846:3 286:4 321846:5 286:9 321846:3 ...


In [70]:
# drop rows with no value
# merged_df.dropna(inplace=True)
# print(len(merged_df))

In [71]:
k2_output_filt["taxon_id"]

0          321846
1          321846
2          321846
3          321846
4          321846
            ...  
1197487       287
1197488       287
1197489       287
1197490       287
1197491       287
Name: taxon_id, Length: 750143, dtype: int64

In [72]:
# Check for discrepancies
# discrepancies = merged_df[merged_df['taxon_id'] != pd.Series(k2_output_filt['taxon_id'])]

# # If there are no discrepancies, discrepancies dataframe will be empty
# if discrepancies.empty:
#     print("All taxon_ids were added correctly.")
# else:
#     print("Discrepancies found:")
#     print(discrepancies)

In [73]:
# Now that we have a DataFrame containing the sequence_ids, sequences, and taxon_ids, we can perform EC analysis with DeepEC
# First, let's create a fasta file containing all the seq_ids and sequences corresponding to a taxon

# Example taxon_id to filter
target_taxon_id = 321846

# Filter rows in merged_df based on the target_taxon_id
filtered_df = merged_df[merged_df['taxon_id'] == target_taxon_id]

# Extract seq_ids and sequences
seq_ids = filtered_df['seq_id'].tolist()
sequences = filtered_df['sequence'].tolist()

# Writing to a FASTA file
fasta_file = '/home/j/jparkin/wongkoji/data/taxon_321846_reads.fasta'

with open(fasta_file, 'w') as fasta_file:
    for seq_id, sequence in zip(seq_ids, sequences):
        fasta_file.write(f">{seq_id}\n{sequence}\n")

In [74]:
merged_df["taxon_id"].unique()

array([ 321846, 1114970,     294,   47883,   76760,     317,   53408,
       2895471,  930166,  587753,  380021, 2745498,     303,     287,
         76758, 2842346,   78327,   76759, 2565368,  107445, 2219225])

In [76]:
parent_dir

'/gpfs/fs1/home/j/jparkin/wongkoji/final_pipeline'

In [84]:
# Now that we have a DataFrame containing the sequence_ids, sequences, and taxon_ids, we can perform EC analysis with DeepEC
# First, let's create a fasta file containing all the seq_ids and sequences corresponding to a taxon

all_taxa = merged_df["taxon_id"].unique()
# Example taxon_id to filter
taxon = 321846

def get_taxon_reads(taxon_id, fasta_file):
    """Filter by taxon id and write to a file fasta_file"""
    # Filter rows in merged_df based on the target_taxon_id
    filtered_df = merged_df[merged_df['taxon_id'] == taxon_id]

    # Extract seq_ids and sequences
    seq_ids = filtered_df['seq_id'].tolist()
    sequences = filtered_df['sequence'].tolist()

    with open(fasta_file, 'w') as fasta_file:
        for seq_id, sequence in zip(seq_ids, sequences):
            fasta_file.write(f">{seq_id}\n{sequence}\n")
    fasta_file.close()

get_taxon_reads(1114970, os.path.join(parent_dir, "tmp/taxon_reads/taxon_1114970_reads.fasta"))

In [80]:
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

def translate_without_internal_stops(seq):
    stop_present = False
    translated_seq = seq.translate()
    if '*' in translated_seq:
        return None
    return str(translated_seq)

def translate_reads(input_file, output_file):
    valid_records = []
    with open(input_file, 'r') as f:
        records = SeqIO.parse(f, 'fasta')
        for record in records:
            translated_seq = translate_without_internal_stops(record.seq)
            if translated_seq:
                protein_record = SeqRecord(Seq(translated_seq), id=record.id, description="")
                valid_records.append(protein_record)

    with open(output_file, 'w') as f:
        SeqIO.write(valid_records, f, 'fasta')

input_file = f'/home/j/jparkin/wongkoji/data/taxon_{taxon}_reads.fasta'
output_file = f'/home/j/jparkin/wongkoji/data/taxon_{taxon}_reads_translated.fasta'
translate_reads(input_file, output_file)

/scratch/j/jparkin/wongkoji/miniconda3/envs/deepectransformer/lib/python3.6/site-packages/Bio/Seq.py:2338: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  BiopythonWarning,


In [85]:
taxon_id_filepath = os.path.join(parent_dir, "tmp/taxon_ids.txt")
with open(taxon_id_filepath, 'w') as taxon_file:
    for taxon in all_taxa:
        path = os.path.join(parent_dir, f"tmp/taxon_reads/taxon_{taxon}_reads.fasta")
        get_taxon_reads(taxon, path)
        output_path = os.path.join(parent_dir, f"tmp/translated_reads/taxon_{taxon}_translated.fasta")
        translate_reads(path, output_path)
        taxon_file.write(f"{taxon}\n")

/scratch/j/jparkin/wongkoji/miniconda3/envs/deepectransformer/lib/python3.6/site-packages/Bio/Seq.py:2338: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  BiopythonWarning,
/scratch/j/jparkin/wongkoji/miniconda3/envs/deepectransformer/lib/python3.6/site-packages/Bio/Seq.py:2338: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  BiopythonWarning,
/scratch/j/jparkin/wongkoji/miniconda3/envs/deepectransformer/lib/python3.6/site-packages/Bio/Seq.py:2338: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  BiopythonWarning,
/scratch/j/jparkin/wongkoji/miniconda3/envs/deepectransformer/lib/python3.6/site-packages/Bio/